In [ ]:
import sys
sys.path.append(r'C:\Users\Gosia\Desktop\FDR-Metabolomics\src\passatutto_parser.py')
import passatutto_parser as pp

In [ ]:
pre_spectrums_query = pp.PassatuttoParser(r'C:\Users\Gosia\Desktop\MassbankOrbi').parse_folder()
pre_spectrums_lib = pp.PassatuttoParser(r'C:\Users\Gosia\Desktop\Gnps_Noise_Filtered').parse_folder()

In [ ]:
from matchms.importing.load_from_json import as_spectrum
spectrums_query = []
for i, s in enumerate( pre_spectrums_query ):
    spectrums_query.append(as_spectrum(s))
    if i and i % 10 == 0:
        print('processed %d', i)
spectrums_lib = []
for i, s in enumerate( pre_spectrums_lib ):
    spectrums_lib.append(as_spectrum(s))
    if i and i % 10 == 0:
        print('processed %d', i)

In [56]:
from matchms.similarity import CosineGreedy
cosine_greedy = CosineGreedy(tolerance=0.2)

In [57]:
from rdkit.Chem.inchi import InchiToInchiKey,MolToInchiKey
def inchisEqual(i1, i2):
    return i1.split("/")[:4] == i2.split("/")[:4]
    #return InchiToInchiKey(i1).split('-')[0] == InchiToInchiKey(i2).split('-')[0]

In [58]:
import bisect
from collections import namedtuple
Hit = namedtuple('Hit', ['query', 'target', 'score', 'hit'])
def get_hits(query_spec, library_spec, precursor_tol = 1, metaKey='parent_mass'):
    library_spec.sort(key = lambda x: x.metadata[metaKey])
    hits = []
    misses = []
    library_prec_list = [x.metadata[metaKey] for x in library_spec]
    for q in query_spec:
        min_mz = q.metadata[metaKey] - precursor_tol
        max_mz = q.metadata[metaKey] + precursor_tol
        pos = bisect.bisect_right(library_prec_list, min_mz)
        pos2 = pos
        if pos2 <= len(library_prec_list) -1:
            while library_prec_list[pos2] < max_mz:
                if pos2 < len(library_prec_list)-1:
                    pos2 += 1 
        if pos == pos2:
            # nothing in precursor range
            misses.append(q)
        else:
            scores = []
            for l in library_spec[pos:pos2]:
                s,_ = cosine_greedy.pair(q,l)
                scores.append((s, l))
            scores.sort(key = lambda x: x[0], reverse=True)
            hits.append(Hit(q, scores[0][0], scores[0][1], inchisEqual(q.metadata['inchi'],scores[0][1].metadata['inchi']))) # query, best cosine score, target, hit_true 
    return hits, misses

In [ ]:
hits, misses = get_hits(spectrums_query, spectrums_lib)

In [ ]:
print(hits)